In [1]:
from mintalib.samples import sample_prices


%load_ext cython

In [2]:
prices = sample_prices()
prices


,open,high,low,close,volume
date,,,,,
1980-12-12,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18,0.091630,0.092061,0.091630,0.091630,73449600
...,...,...,...,...,...
2024-09-12,222.500000,223.550003,219.820007,222.770004,37498200
2024-09-13,223.580002,224.039993,221.910004,222.500000,36766600
2024-09-16,216.539993,217.220001,213.919998,216.320007,59357400


In [3]:
# Definitions to remove missing definitions warnings

def np_sma(xs, period: int):
    pass

def calc_sma(series, period: int, flag: bool=False):
    pass


In [4]:
%%cython -c=-Wno-unreachable-code

import numpy as np

from libc.math cimport isnan

from mintalib.core import wrap_result

cdef double NAN = float('nan')


def calc_sma(series, long period, bint wrap = False):
    """ simple moving average """

    cdef double[:] xs = np.asarray(series, float)
    cdef long size = xs.size

    cdef object result = np.full(size, np.nan)
    cdef double[:] output = result

    cdef double v = NAN, rsum = 0
    cdef long i = 0, j = 0, count = 0

    for i in range(size):
        v = xs[i]

        if not isnan(v):
            rsum += v
            count += 1

        while count > period and j < i:
            v, j = xs[j], j+1
            if not isnan(v):
                rsum -= v
                count -= 1

        if count >= period:
            output[i] = rsum / count

    if wrap:
        result = wrap_result(result, series)

    return result

print("done!")

done!


In [5]:
calc_sma(prices.close, 5)

array([         nan,          nan,          nan, ..., 220.87200317,
       220.20800171, 219.81400146])

In [6]:
%%cython -c=-Wno-unreachable-code

import numpy as np

from libc.math cimport isnan

from mintalib.core import wrap_result

cdef double NAN = float('nan')


cpdef np_sma(double[:] xs, long period):
    """ simple moving average """

    cdef long size = xs.size

    cdef object result = np.full(size, np.nan)
    cdef double[:] output = result

    cdef double v = NAN, rsum = 0
    cdef long i = 0, j = 0, count = 0

    for i in range(size):
        v = xs[i]

        if not isnan(v):
            rsum += v
            count += 1

        while count > period and j < i:
            v, j = xs[j], j+1
            if not isnan(v):
                rsum -= v
                count -= 1

        if count >= period:
            output[i] = rsum / count

    # also return output.base
    return result 


def calc_sma(series, int period, bint wrap = False):
    """ simple moving average """

    cdef double[:] xs = np.asarray(series, float)
    cdef object result = np_sma(xs, period)

    if wrap:
        result = wrap_result(result, series)

    return result

print("done!")

done!


In [7]:
res = np_sma(prices.close.values, 5)
print(type(res))
res

<class 'numpy.ndarray'>


array([         nan,          nan,          nan, ..., 220.87200317,
       220.20800171, 219.81400146])

In [8]:
res = calc_sma(prices.close, 3)
print(type(res))
res

<class 'numpy.ndarray'>


array([           nan,            nan, 9.32074810e-02, ...,
       2.20530004e+02, 2.18536667e+02, 2.17933334e+02])

In [9]:
res = calc_sma(prices.close, 3, wrap=True)
print(type(res))
res

<class 'pandas.core.series.Series'>


date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16      0.093207
1980-12-17      0.089909
1980-12-18      0.089192
                 ...    
2024-09-12    221.846670
2024-09-13    222.643336
2024-09-16    220.530004
2024-09-17    218.536667
2024-09-18    217.933334
Length: 11033, dtype: float64